In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [3]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))


[]


In [ ]:
df = pd.read_csv('nifty_dataPEE.csv')

df.set_index('lasttradetime', inplace=True)
scaler = MinMaxScaler(feature_range=(0,1))
df_scaled = scaler.fit_transform(df)
lookback_window = 5
features = df.shape[1]
def create_sequences(data, lookback_window):
  X, y = [], []
  for i in range(len(data) - lookback_window - 1):
    X.append(data[i:(i + lookback_window), :])
    y.append(data[i + lookback_window, :])
  return np.array(X), np.array(y)

X, y = create_sequences(df_scaled, lookback_window)
X_train, y_train = X, y  # Replace with train/validation split if desired
model = Sequential()
model.add(LSTM(units=500, return_sequences=True, input_shape=(lookback_window, features)))
model.add(LSTM(units=100))
model.add(Dense(units=features))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=150,batch_size=30)
number_of_predictions = 50
forecasted_values = []
last_sequence = X_train[-1]

for _ in range(number_of_predictions):
  next_data_point = model.predict(np.expand_dims(last_sequence, axis=0))
  next_data_point_original = scaler.inverse_transform(next_data_point)
  forecasted_values.append(next_data_point_original.flatten())
  last_sequence = np.append(last_sequence[1:], next_data_point, axis=0)

forecast_df = pd.DataFrame(forecasted_values, columns=df.columns)
print(forecast_df['close'].max(),forecast_df['close'].min())


In [6]:
import numpy as np
import tensorflow as tf

# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [11]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
lookback_window =3
features = 5
def create_sequences(data, lookback_window):
    X, y = [], []
    for i in range(len(data) - lookback_window - 1): 
        X.append(data[i:(i + lookback_window), :])
        y.append(data[i + lookback_window, 0:5]) 
    return np.array(X), np.array(y)
model = Sequential()
model.add(LSTM(units=200, return_sequences=True, input_shape=(lookback_window, features)))
model.add(LSTM(units=200))
model.add(Dense(200, activation=tf.nn.leaky_relu))
model.add(Dense(200, activation=tf.nn.leaky_relu))
model.add(Dense(units=5)) 
model.compile(loss='mean_squared_error', optimizer='adam')
X, y = create_sequences(df_scaled, lookback_window)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


model.fit(X_train, y_train, epochs=50, batch_size=30, validation_data=(X_test, y_test))
y_pred = model.predict(X_test)
y_pred_original = scaler.inverse_transform(y_pred)
data = pd.DataFrame(y_pred_original, columns=['open', 'high', 'low' ,'close','volume'])
data 
number_of_predictions = 10
forecasted_values = []
last_sequence = X_train[-1]
for _ in range(number_of_predictions):
  next_data_point = model.predict(np.expand_dims(last_sequence, axis=0))
  next_data_point_original = scaler.inverse_transform(next_data_point)
  forecasted_values.append(next_data_point_original.flatten())
  last_sequence = np.append(last_sequence[1:], next_data_point, axis=0)
forecast_df = pd.DataFrame(forecasted_values, columns=df.columns)
print(forecast_df['close'].max(),forecast_df['close'].min())

c:\Users\santh\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 105ms/step - loss: 0.1644 - val_loss: 0.0545
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0307 - val_loss: 0.0458
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 0.0236 - val_loss: 0.0194
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.0144 - val_loss: 0.0117
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0116 - val_loss: 0.0129
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0095 - val_loss: 0.0074
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0082 - val_loss: 0.0086
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0070 - val_loss: 0.0084
Epoch 9/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.0065 - val_loss: 0.0096
Epoch 10/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.0064 - val_loss: 0.0083
Epoch 11/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.0063 - val_loss: 0.0082
Epoch 12/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.0062 - val_loss: 0.0080
